In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,LeakyReLU
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow as tf

import json

In [ ]:
with open('data/mapping.json') as f:
    CLASS_MAPPING = json.load(f)

In [ ]:
train = pd.read_csv('data/emnist-balanced-train.csv',header=None)
test = pd.read_csv('data/emnist-balanced-test.csv',header=None)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
#28*28 image
train.head()

In [ ]:
len(pd.unique(train[0]))

In [ ]:
len(pd.unique(test[0]))

In [ ]:
X_train = train.drop(0,axis=1)
y_train = train[0]
X_train = X_train.values.reshape((len(X_train),28,28,1))

In [ ]:
X_test = test.drop(0,axis=1)
y_test = test[0]
X_test = X_test.values.reshape((len(X_test),28,28,1))

In [ ]:
plt.imshow(X_train[6].transpose().reshape(28,28),cmap='gray')
print(CLASS_MAPPING[str(y_train[6])])

In [ ]:
batch_size = 128
num_classes = len(pd.unique(train[0]))
epochs = 15

In [ ]:
#28x28 pixel images
img_rows,img_cols = 28,28

In [ ]:
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0],1,img_rows,img_cols)
    X_test = X_test.reshape(X_test.shape[0],1,img_rows,img_cols)
    input_shape = (1,img_rows,img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0],img_rows,img_cols,1)
    X_test = X_test.reshape(X_test.shape[0],img_rows,img_cols,1)
    input_shape = (img_rows,img_cols,1)  
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


X_train_tmp = []
for x in X_train:
    X_train_tmp.append(x.transpose().reshape(28,28,1))
X_train= np.array(X_train_tmp)

X_test_tmp = []
for x in X_test:
    X_test_tmp.append(x.transpose().reshape(28,28,1))
X_test= np.array(X_test_tmp)


y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)


print("X_train shape: "+ str(X_train.shape))
print("X_test shape: "+ str(X_test.shape))

In [ ]:
def base_model(num_classes,input_shape):
    
    model = Sequential()
    
    model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape))
    model.add(Conv2D(64,(3,3),activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(128,activation='relu'))
    
    model.add(Dropout(0.5))
    
    model.add(Dense(num_classes,activation='softmax'))
    
    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer = keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    
    return model 

In [ ]:
def model_v1(num_classes,input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D((2, 2),padding='same'))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
    model.add(LeakyReLU(alpha=0.1))                  
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.1))           
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    
    #model.load_weights("model/model_v1_best_weights.h5")
    model.compile(loss=keras.losses.categorical_crossentropy, 
                      optimizer = keras.optimizers.adam(),
                      metrics=['accuracy'])
    return model

In [ ]:
def model_vgg16(num_classes,input_shape):
    model = Sequential()
    
    model.add(Conv2D(64, (3, 3), input_shape=input_shape, padding='same',
           activation='relu'))  
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))    
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same',))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same',))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same',))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same',))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    
    model.add(Flatten())
    
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    #model.load_weights("model/model_vgg16_best_weights.h5")
    
    model.compile(loss=keras.losses.categorical_crossentropy, 
                      optimizer = keras.optimizers.adam(),
                      metrics=['accuracy'])
    return model


In [ ]:
model = model_v1(num_classes,input_shape)

# checkpoint
filepath="model/model_v1_best_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.fit(X_train,y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks_list,
          validation_data=(X_test,y_test))

In [ ]:
model = model_vgg16(num_classes,input_shape)

# checkpoint
filepath="model/model_vgg16_best_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.fit(X_train,y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks_list,
          validation_data=(X_test,y_test))

In [ ]:
MaxPooling2D()

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model_json = model.to_json()
with open('model/model.json','w') as f:
    f.write(model_json)
model.save_weights('model/model.h5')

## Transfer Learning

In [ ]:
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
import cv2
import pickle

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
#Initial Shape
X_train[0].shape

In [ ]:
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:
NB_IV3_LAYERS_TO_FREEZE = 172

In [ ]:
resize = cv2.resize(X_train[1], (150,150))
img = cv2.cvtColor(resize,cv2.COLOR_GRAY2RGB)

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
def preprocess_img(img):
    resize = cv2.resize(img, (IMG_HEIGHT,IMG_WIDTH))
    rgb_img = cv2.cvtColor(resize,cv2.COLOR_GRAY2RGB)
    return rgb_img

In [ ]:
def get_every_n(a, n):
    for i in range(a.shape[0] // n):
        yield a[n*i:n*(i+1)]

X_train_imgs = []
for x_train_tmp in get_every_n(X_train,100):
    for tmp_img in x_train_tmp:
        X_train_imgs.append(preprocess_img(tmp_img))

In [ ]:
with open('data/X_train_imgs_150_150.pkl','w') as f:
    pickle.dump(X_train_imgs,f)

In [ ]:
def add_new_last_layer(base_model, nb_classes):
  """
  Add last layer to the convnet
  Args:
    base_model: keras model excluding top
    nb_classes: # of classes
  Returns:
    new keras model with last layer
  """
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x) 
  predictions = Dense(nb_classes, activation='softmax')(x) 
  model = Model(input=base_model.input, output=predictions)
  return model

In [ ]:
def setup_to_transfer_learn(model, base_model):
  """Freeze all layers and compile the model"""
  for layer in base_model.layers:
    layer.trainable = False
  model.compile(optimizer='rmsprop',    
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

In [ ]:
def setup_to_finetune(model):
   """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top 
      layers.
   note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in 
         the inceptionv3 architecture
   Args:
     model: keras model
   """
   for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
      layer.trainable = False
   for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
      layer.trainable = True
   model.compile(optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.9),   
                 loss='categorical_crossentropy')
   return model

In [ ]:
plt.imshow(preprocess_img(X_train[0]))

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_img)

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

In [ ]:
model = add_new_last_layer(base_model, num_classes)

In [ ]:
history = model.fit(X_train_imgs,y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_train_imgs,y_train))

In [ ]:
model =setup_to_finetune(model)